# Introduction


We are using LLama3 to write code.

# Preparation

In [1]:
from time import time
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import display, Markdown

In [2]:
model = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

2024-05-13 14:17:37.051371: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-13 14:17:37.051523: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-13 14:17:37.165244: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Test function

Let's define the query model function.

In [3]:
def query_model(
        system_message,
        user_message,
        temperature=0.7,
        max_length=1024
        ):
    start_time = time()
    user_message = "Question: " + user_message + " Answer:"
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
        ]
    prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
        )
    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    sequences = pipeline(
        prompt,
        do_sample=True,
        top_p=0.9,
        temperature=temperature,
        #num_return_sequences=1,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        pad_token_id=pipeline.model.config.eos_token_id
    )
    #answer = f"{sequences[0]['generated_text'][len(prompt):]}\n"
    answer = sequences[0]['generated_text']
    end_time = time()
    ttime = f"Total time: {round(end_time-start_time, 2)} sec."

    return user_message + " " + answer  + " " +  ttime


We also define an utility function for displaying the results.

In [4]:
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

# Python code

Let's start testing the model with some Python questions.

In [5]:
system_message = """
You are an AI assistant designed to write simple Python code.
Please answer with the listing of the Python code.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a function in Python to calculate the area of a rectangle with edges of L and l",
    temperature=0.1,
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in Python to calculate the area of a rectangle with edges of L and l 

**<font color='green'>Answer:</font>** Here is the Python code for the function:

```
def rectangle_area(L, l):
    return L * l
``` 

**<font color='magenta'>Total time:</font>** 4.24 sec.

In [6]:
response = query_model(
    system_message,
    user_message="Please write a function in Python to order a list",
    temperature=0.1,
    max_length=512)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in Python to order a list 

**<font color='green'>Answer:</font>** Here is a Python function that orders a list:

```
def order_list(lst):
    return sorted(lst)
``` 

**<font color='magenta'>Total time:</font>** 1.68 sec.

In [7]:
response = query_model(
    system_message,
    user_message="Please write a Python module for a phone agenda.", 
    temperature=0.1,
    max_length=1024)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a Python module for a phone agenda. 

**<font color='green'>Answer:</font>** Here is a simple Python module for a phone agenda:

```
class PhoneAgenda:
    def __init__(self):
        self.contacts = {}

    def add_contact(self, name, phone_number):
        self.contacts[name] = phone_number

    def delete_contact(self, name):
        if name in self.contacts:
            del self.contacts[name]
        else:
            print("Contact not found")

    def update_contact(self, name, phone_number):
        if name in self.contacts:
            self.contacts[name] = phone_number
        else:
            print("Contact not found")

    def search_contact(self, name):
        if name in self.contacts:
            return self.contacts[name]
        else:
            return "Contact not found"

    def display_contacts(self):
        for name, phone_number in self.contacts.items():
            print(f"Name: {name}, Phone Number: {phone_number}")


def main():
    agenda = PhoneAgenda()

    while True:
        print("\nPhone Agenda Menu:")
        print("1. Add Contact")
        print("2. Delete Contact")
        print("3. Update Contact")
        print("4. Search Contact")
        print("5. Display Contacts")
        print("6. Exit")

        choice = input("Enter your choice: ")

        if choice == "1":
            name = input("Enter contact name: ")
            phone_number = input("Enter contact phone number: ")
            agenda.add_contact(name, phone_number)
        elif choice == "2":
            name = input("Enter contact name: ")
            agenda.delete_contact(name)
        elif choice == "3":
            name = input("Enter contact name: ")
            phone_number = input("Enter new contact phone number: ")
            agenda.update_contact(name, phone_number)
        elif choice == "4":
            name = input("Enter contact name: ")
            print(agenda.search_contact(name))
        elif choice == "5":
            agenda.display_contacts()
        elif choice == "6":
            break
        else:
            print("Invalid choice. Please choose a valid option.")


if __name__ == "__main__":
    main()
``` 

**<font color='magenta'>Total time:</font>** 30.22 sec.

In [8]:
response = query_model(
    system_message,
    user_message="""
                            Please write a Python service using Flask to expose a machine learning model. 
                            The service has one endpoint (POST) that receives three features:
                            - area
                            - number of rooms
                            - number of bathrooms
                            The model returns the price of the property.""",
    temperature=0.1,
    max_length=512)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
                            Please write a Python service using Flask to expose a machine learning model. 
                            The service has one endpoint (POST) that receives three features:
                            - area
                            - number of rooms
                            - number of bathrooms
                            The model returns the price of the property. 

**<font color='green'>Answer:</font>** Here is a simple Python service using Flask to expose a machine learning model:

```
from flask import Flask, request, jsonify
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

app = Flask(__name__)

# Load the dataset
df = pd.read_csv('property_data.csv')

# Split the data into features and target
X = df[['area', 'number of rooms', 'number of bathrooms']]
y = df['price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Define the endpoint
@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    area = data['area']
    number_of_rooms = data['number_of_rooms']
    number_of_bathrooms = data['number_of_bathrooms']

    # Scale the input data
    scaler = StandardScaler()
    input_data = scaler.transform([[area, number_of_rooms, number_of_bathrooms]])

    # Make the prediction
    prediction = model.predict(input_data)

    return jsonify({'price': prediction[0]})

if __name__ == '__main__':
    app.run(debug=True)
``` 

**<font color='magenta'>Total time:</font>** 22.37 sec.

# C++ code

Let's continue with few C++ questions.

In [9]:
system_message = """
You are an AI assistant designed to write simple C++ code.
Please answer with the listing of the C++ code.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a function in C++ to calculate the area of a rectangle with edges of L and l",
    temperature=0.1,
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in C++ to calculate the area of a rectangle with edges of L and l 

**<font color='green'>Answer:</font>** Here is the C++ code for the function:

```cpp
#include<iostream>
using namespace std;

float area(float L, float l) {
    return L * l;
}

int main() {
    float L, l;
    cout << "Enter the length of the rectangle: ";
    cin >> L;
    cout << "Enter the breadth of the rectangle: ";
    cin >> l;
    cout << "The area of the rectangle is: " << area(L, l);
    return 0;
}
``` 

**<font color='magenta'>Total time:</font>** 7.35 sec.

In [10]:
response = query_model(
    system_message,
    user_message="Please write a function in C++ to traverse a balanced tree",
    temperature=0.1,
    max_length=512)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in C++ to traverse a balanced tree 

**<font color='green'>Answer:</font>** Here is a simple C++ function to traverse a balanced binary tree in-order, pre-order, and post-order:

```cpp
#include <iostream>
using namespace std;

// Define the structure for a node in the tree
struct Node {
    int data;
    Node* left;
    Node* right;
};

// Function to create a new node
Node* createNode(int data) {
    Node* newNode = new Node();
    if (!newNode) {
        cout << "Memory error\n";
        return NULL;
    }
    newNode->data = data;
    newNode->left = newNode->right = NULL;
    return newNode;
}

// Function to perform in-order traversal
void inOrderTraversal(Node* root) {
    if (root == NULL) return;
    inOrderTraversal(root->left);
    cout << root->data << " ";
    inOrderTraversal(root->right);
}

// Function to perform pre-order traversal
void preOrderTraversal(Node* root) {
    if (root == NULL) return;
    cout << root->data << " ";
    preOrderTraversal(root->left);
    preOrderTraversal(root->right);
}

// Function to perform post-order traversal
void postOrderTraversal(Node* root) {
    if (root == NULL) return;
    postOrderTraversal(root->left);
    postOrderTraversal(root->right);
    cout << root->data << " ";
}

int main() {
    Node* root = createNode(1);
    root->left = createNode(2);
    root->right = createNode(3);
    root->left->left = createNode(4);
    root->left->right = createNode(5);

    cout << "In-order traversal: ";
    inOrderTraversal(root);
    cout << endl;

    cout << "Pre-order traversal: ";
    preOrderTraversal(root);
    cout << endl;

    cout << "Post-order traversal: ";
    postOrderTraversal(root);
    cout << endl;

    return 0;
}
``` 

**<font color='magenta'>Total time:</font>** 28.53 sec.

In [11]:
response = query_model(
    system_message,
    user_message="Please write a class in C++ for allocation of small objects",
    temperature=0.1,
    max_length=512)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a class in C++ for allocation of small objects 

**<font color='green'>Answer:</font>** Here is a simple implementation of a class in C++ for allocation of small objects using a pool allocator:

```cpp
#include <iostream>
#include <vector>
#include <algorithm>

class SmallObjectAllocator {
private:
    std::vector<char> pool;
    size_t objectSize;

public:
    SmallObjectAllocator(size_t objectSize) : objectSize(objectSize) {
        pool.resize(objectSize, 0);
    }

    void* allocate() {
        for (char& byte : pool) {
            if (byte == 0) {
                byte = 1;
                return &byte;
            }
        }
        return nullptr;
    }

    void deallocate(void* ptr) {
        if (ptr!= nullptr) {
            char* charPtr = static_cast<char*>(ptr);
            *charPtr = 0;
        }
    }
};

int main() {
    SmallObjectAllocator allocator(16); // Allocate objects of size 16 bytes

    void* obj1 = allocator.allocate();
    void* obj2 = allocator.allocate();
    void* obj3 = allocator.allocate();

    // Use the allocated objects
    //...

    allocator.deallocate(obj1);
    allocator.deallocate(obj2);
    allocator.deallocate(obj3);

    return 0;
}
```

This code creates a pool of bytes of a specified size, and provides methods to allocate and deallocate objects of that size from the pool. The `allocate` method returns a pointer to the next available object in the pool, and the `deallocate` method marks an object as available again. 

**<font color='magenta'>Total time:</font>** 22.12 sec.

# SQL code

Let's continue by generating some transactional SQL.

In [12]:
system_message = """
You are an AI assistant designed to write simple SQL code.
Please answer with the listing of the SQL code.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a small stored procedure in SQL to debit one account and credit another account.",
    temperature=0.1,
    max_length=512)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a small stored procedure in SQL to debit one account and credit another account. 

**<font color='green'>Answer:</font>** Here is a sample stored procedure in SQL to debit one account and credit another account:

```
CREATE PROCEDURE DebitAndCredit
    @account_to_debit int,
    @account_to_credit int,
    @amount decimal(10,2)
AS
BEGIN
    BEGIN TRANSACTION
    BEGIN TRY
        UPDATE Accounts
        SET Balance = Balance - @amount
        WHERE AccountID = @account_to_debit

        UPDATE Accounts
        SET Balance = Balance + @amount
        WHERE AccountID = @account_to_credit

        COMMIT TRANSACTION
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION
        THROW
    END CATCH
END
``` 

**<font color='magenta'>Total time:</font>** 9.74 sec.

# Erlang code


Let's continue by generating some Erlang code.

In [13]:
system_message = """
You are an AI assistant designed to write simple Erlang code.
Please answer with the listing of the Erlang code.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a small module in Erlang for sending transactions between two parties.",
    temperature=0.1,
    max_length=512)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a small module in Erlang for sending transactions between two parties. 

**<font color='green'>Answer:</font>** Here is a simple Erlang module that simulates sending transactions between two parties:

```erlang
-module(transaction).
-export([start/0, send/2, receive/1]).

% Initialize the balance for both parties
party1_balance = 1000.
party2_balance = 0.

start() ->
    io:format("Transaction system started.~n").

send(Party, Amount) ->
    case Party of
        party1 ->
            if Amount =< party1_balance ->
                party1_balance = party1_balance - Amount,
                party2_balance = party2_balance + Amount,
                io:format("Party 1 sent ~p to Party 2.~n", [Amount]);
               _ ->
                io:format("Party 1 does not have enough balance.~n")
            end;
        party2 ->
            io:format("Party 2 cannot send transactions.~n");
        _ ->
            io:format("Invalid party.~n")
    end.

receive(Party) ->
    case Party of
        party2 ->
            io:format("Party 2 received ~p from Party 1.~n", [party2_balance]);
        _ ->
            io:format("Invalid party.~n")
    end.
``` 

**<font color='magenta'>Total time:</font>** 17.99 sec.

# Conclusion

Llama3 proved to be proficient while writing code in a large number of languages, including objectual code, procedural, functional, and directive.